In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/Shareddrives/Audio-Recorder/uploads/

/content/gdrive/Shareddrives/Audio-Recorder/uploads


In [4]:
!pip install azure-cognitiveservices-speech

     |████████████████████████████████| 3.1 MB 5.4 MB/s 


In [5]:
import azure.cognitiveservices.speech as speechsdk
import glob, os #To access audio files from drive

In [6]:
## API Keys for Azure

subscription_key = "326c05844287429f904bff80816015e9"
region = "eastus"

In [7]:
def audioRecognizer(audioFile, myList):
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    speech_config.output_format=speechsdk.OutputFormat.Detailed
    audio_input = speechsdk.AudioConfig(filename=audioFile)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
    
    result = speech_recognizer.recognize_once_async().get()

    myList.append(result.json)
    print(result.json)

In [8]:
%ls

4ca5a1c9-6c6b-4ad7-848d-bb06210494f6/  8c08d402-bfe4-4af4-a844-53dccd158927/
7049d9bd-fda2-413e-af00-ac50a3e9b463/  8e38cb31-2189-4845-a9f5-e5076466672d/
7cc200a5-367b-411f-ac56-c64f563084aa/  c914fe94-b1e3-4262-9b3a-a4bc18357c54/


In [9]:
cwd = os.getcwd()
TotalDict = {}
for user in os.listdir(): # list of users
  userDict = {}
  if user.__contains__("7049d9bd-fda2-413e-af00-ac50a3e9b463"):
    for command in glob.glob(os.path.join(cwd,user, "*/")):
      if(os.path.isdir(command)):
        commandList = []
        for file in glob.glob(os.path.join(cwd,command,"*.wav")):
          audioRecognizer(file,commandList)
      userDict[command] = commandList
    TotalDict[user] = userDict

{"Id":"4864fe65d62d4004b3712e51f6f2a36d","RecognitionStatus":"Success","Offset":3700000,"Duration":13100000,"DisplayText":"Show confirmation.","NBest":[{"Confidence":0.9161852,"Lexical":"show confirmation","ITN":"show confirmation","MaskedITN":"show confirmation","Display":"Show confirmation."},{"Confidence":0.89413786,"Lexical":"show confirmations","ITN":"show confirmations","MaskedITN":"show confirmations","Display":"show confirmations"},{"Confidence":0.75852233,"Lexical":"sho confirmation","ITN":"sho confirmation","MaskedITN":"sho confirmation","Display":"sho confirmation"},{"Confidence":0.8537973,"Lexical":"shell confirmation","ITN":"shell confirmation","MaskedITN":"shell confirmation","Display":"shell confirmation"},{"Confidence":0.8671833,"Lexical":"sure confirmation","ITN":"sure confirmation","MaskedITN":"sure confirmation","Display":"sure confirmation"}]}
{"Id":"6716311dbf61465da1cf2086cc9c9df8","RecognitionStatus":"Success","Offset":2000000,"Duration":13500000,"DisplayText":"S

In [10]:
TotalDict

{'7049d9bd-fda2-413e-af00-ac50a3e9b463': {'/content/gdrive/Shareddrives/Audio-Recorder/uploads/7049d9bd-fda2-413e-af00-ac50a3e9b463/Navigate to place/': ['{"Id":"9a4279a020114069838a04389388b3c5","RecognitionStatus":"Success","Offset":3200000,"Duration":15300000,"DisplayText":"Navigate your place.","NBest":[{"Confidence":0.84200305,"Lexical":"navigate your place","ITN":"navigate your place","MaskedITN":"navigate your place","Display":"Navigate your place."},{"Confidence":0.83034945,"Lexical":"navigate to your place","ITN":"navigate to your place","MaskedITN":"navigate to your place","Display":"navigate to your place"},{"Confidence":0.84823376,"Lexical":"navigate to place","ITN":"navigate to place","MaskedITN":"navigate to place","Display":"navigate to place"},{"Confidence":0.83989877,"Lexical":"navigate to replace","ITN":"navigate to replace","MaskedITN":"navigate to replace","Display":"navigate to replace"},{"Confidence":0.84337205,"Lexical":"navigate you place","ITN":"navigate you pl

In [11]:
FinalTotalDict = {}
for user in TotalDict.keys():
  userDict = {}
  for command in TotalDict[user].keys():
    command_ = command[command.find(user) + len(user): ][1:-1]
    userDict[command_] = TotalDict[user][command]
  FinalTotalDict[user] = userDict

In [12]:
FinalTotalDict

{'7049d9bd-fda2-413e-af00-ac50a3e9b463': {'Navigate to place': ['{"Id":"9a4279a020114069838a04389388b3c5","RecognitionStatus":"Success","Offset":3200000,"Duration":15300000,"DisplayText":"Navigate your place.","NBest":[{"Confidence":0.84200305,"Lexical":"navigate your place","ITN":"navigate your place","MaskedITN":"navigate your place","Display":"Navigate your place."},{"Confidence":0.83034945,"Lexical":"navigate to your place","ITN":"navigate to your place","MaskedITN":"navigate to your place","Display":"navigate to your place"},{"Confidence":0.84823376,"Lexical":"navigate to place","ITN":"navigate to place","MaskedITN":"navigate to place","Display":"navigate to place"},{"Confidence":0.83989877,"Lexical":"navigate to replace","ITN":"navigate to replace","MaskedITN":"navigate to replace","Display":"navigate to replace"},{"Confidence":0.84337205,"Lexical":"navigate you place","ITN":"navigate you place","MaskedITN":"navigate you place","Display":"navigate you place"}]}',
   '{"Id":"bdb0c

In [13]:
import ast
CountDict = {}
for user in FinalTotalDict.keys():
  count = 0
  for command in FinalTotalDict[user].keys():
    trimmedCommand = command.strip().lower()
    for trial in FinalTotalDict[user][command]:
      if len(trial)!= 0:
        trial = ast.literal_eval(trial)
        if trial['RecognitionStatus'] == "Success":
          speechResult = trial["NBest"][0]["Lexical"].lower()
          if speechResult == trimmedCommand:
            count+=1
    
  CountDict[user] = count

In [14]:
CountDict

{'7049d9bd-fda2-413e-af00-ac50a3e9b463': 79}

In [15]:
sum = 0
for count in CountDict.keys():
  sum+= CountDict[count]

total = 96 * len(CountDict)

## (Total number of audio files  = 96 x # of users)

print("Accuracy : " , sum/total)

Accuracy :  0.8229166666666666
